# Check if the packet loss data is abnormal

###This notebook finds out all the links which have at least five packet loss measurements in the past one hour and the average value of the packet loss measurements is greater than 2%. It is run by a cron job every hour, and it will write the detailed information of every alarm into Elastic Search with the _index: alarms-year.month and _type: packetloss.

### import all the packages needed for this task

In [1]:
from elasticsearch import Elasticsearch, exceptions as es_exceptions, helpers
import sys
import datetime

### establish the Elastic Search connection

In [2]:
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

### define functions to write an alarm record into ES with detailed info

In [3]:
ipSite={}  # for mapping IP to Site name
toAlertOn=[]

def generate_doc(src_site_ip, dest_site_ip, measurements, avgpl):
   if src_site_ip not in ipSite:
        print('serious source mapping issue')
        return
   if dest_site_ip not in ipSite:
        print('serious destination mapping issue')
        return

   doc = {
        '_index':get_index_name(),
        '_type' : 'packetloss',
        'src' : src_site_ip,
        'dest' : dest_site_ip,
        'srcSite' : ipSite[src_site_ip],
        'destSite' : ipSite[dest_site_ip],
        'alarmTime' : int( (datetime.datetime.utcnow()-datetime.datetime(1970,1,1) ).total_seconds() * 1000 ),
        'measurements' : measurements,
        'packetLossAvg' : avgpl
   }
   return doc

def get_index_name():
    date = datetime.datetime.utcnow().strftime("%Y.%m")   # date format is yyyy.mm
    index_name = 'alarms-'+date
    return index_name



### list all network\_weather* indices

In [4]:
indices = es.cat.indices(index="network_weather_2-*", h="index", request_timeout=600).split('\n')
indices = [x for x in indices if x != '']
indices = [x.strip() for x in indices]
#print(indices)

### find indices to be used

In [5]:

cday  = datetime.datetime.utcnow()
pday  = cday - datetime.timedelta(days=1)
ind1 = 'network_weather_2-%d.%d.%d' % (cday.year, cday.month, cday.day)
ind2 = 'network_weather_2-%d.%d.%d' % (pday.year, pday.month, pday.day)

print ('checking for indices:', ind1, ind2)

ind=[]
if ind1 in indices :
   ind.append(ind1)
if ind2 in indices and cday.hour<3:
   ind.append(ind2)

if len(ind)==0:
   print ('no current indices found. Aborting.')
   sys.exit(1)
else:
   print('will use indices:', ind)

checking for indices: network_weather_2-2016.8.15 network_weather_2-2016.8.14
will use indices: ['network_weather_2-2016.8.15']


### get aggregated data for the past one hour
This query is composed of 3 parts: a) filter - takes only packet loss data, and production servers in last 1h. b) aggregation -  finds average packet loss per source and destination c) finds IP to site name mapping (both source and destination) 

In [6]:
query={
   "size": 0,
   "query": {
    "bool": {
      "must": [
        {"term": { "_type" : "packet_loss_rate"}},
        {"term": { "srcProduction" : True }},
        {"term": { "destProduction" : True }}
      ],
      "filter" : {
        "range" : {
          "timestamp" : {
            "gt": "now-1h"
          }
        }
      }
    }
   },
    "aggs" : {
      "src" : {
        "terms" : { "field" : "src", "size": 1000 },
        "aggs" : {
          "dest" : {
            "terms" : {"field" : "dest", "size": 1000},
            "aggs" : {
              "avgpl" : {
                "avg" :{
                  "field" : "packet_loss"
              }
            }
          }
        }
      }
    },
    "srcSites" : {
      "terms" : { "field" : "src", "size": 1000 },
        "aggs" : {
          "srcsitename" : {
            "terms" : { "field" : "srcSite" }
        }
      }
    },
    "destSites" : {
      "terms" : { "field" : "dest", "size": 1000 },
        "aggs" : {
          "destsitename" : {
            "terms" : { "field" : "destSite" }
        }
      }
    }
  }
}

### execute query

In [7]:
res = es.search(index=ind, body=query, request_timeout=120)
#print(res)

### proces IP to site name mapping data

In [8]:
srcsites=res['aggregations']['srcSites']['buckets']
#print(srcsites)
for sS in srcsites:
   #print(sS)
   siteName=sS['srcsitename']['buckets']
   if len(siteName)==0:
      siteName='UnknownSite'
   else:
      siteName=siteName[0]['key']
   ipSite[sS['key']]=siteName

destsites=res['aggregations']['destSites']['buckets']
#print(destsites)
for dS in destsites:
   #print(dS)
   siteName=dS['destsitename']['buckets']
   if len(siteName)==0:
      siteName='UnknownSite'
   else:
      siteName=siteName[0]['key']
   ipSite[dS['key']]=siteName

print(ipSite)


{'152.84.101.141': 'UnknownSite', '132.206.245.252': 'CA-MCGILL-CLUMEQ-T2', '90.147.66.50': 'UnknownSite', '117.103.105.191': 'Taiwan-LCG2', '18.12.1.171': 'UnknownSite', '134.158.132.200': 'GRIF', '192.101.161.186': 'UnknownSite', '85.122.31.74': 'RO-16-UAIC', '164.58.29.121': 'LUCILLE', '134.75.125.241': 'UnknownSite', '141.108.35.18': 'INFN-ROMA1', '64.57.30.149': 'UnknownSite', '144.92.180.75': 'GLOW', '193.205.76.76': 'UnknownSite', '194.80.35.169': 'UKI-NORTHGRID-LANCS-HEP', '192.12.15.26': 'BNL-ATLAS', '159.93.229.151': 'JINR-T1', '193.206.93.45': 'UnknownSite', '200.136.80.20': 'SPRACE', '109.105.125.232': 'FI_HIP_T2', '134.158.159.85': 'GRIF', '134.219.225.13': 'UKI-LT2-RHUL', '72.36.96.4': 'MWT2', '109.105.124.86': 'NDGF-T1', '149.165.224.247': 'UnknownSite', '134.158.123.183': 'IN2P3-LPC', '157.82.112.68': 'TOKYO-LCG2', '72.36.96.15': 'MWT2', '131.111.66.196': 'UKI-SOUTHGRID-CAM-HEP', '147.231.25.192': 'praguelcg2', '142.150.19.61': 'CA-SCINET-T2', '192.41.230.61': 'UnknownS

### process packet loss averages

In [9]:
src=res['aggregations']['src']['buckets']
#print(src)

for s in src:
   #print(s)
   source=s['key']
   for d in s['dest']['buckets']:
      destination=d['key']
      avgpl=d['avgpl']['value']
      docs=d['doc_count']
#      print(source, destination, docs, avgpl)
      if avgpl > 0.02 and docs > 4:
         toAlertOn.append(generate_doc(source, destination, docs, avgpl))

for alert in toAlertOn:
   print(alert)

{'dest': '161.116.81.235', 'destSite': 'UnknownSite', '_type': 'packetloss', 'packetLossAvg': 1.0, 'alarmTime': 1471281866452, '_index': 'alarms-2016.08', 'srcSite': 'FZK-LCG2', 'src': '192.108.47.12', 'measurements': 11}
{'dest': '194.85.69.75', 'destSite': 'ITEP', '_type': 'packetloss', 'packetLossAvg': 0.05124476795102465, 'alarmTime': 1471281866452, '_index': 'alarms-2016.08', 'srcSite': 'FZK-LCG2', 'src': '192.108.47.12', 'measurements': 7}
{'dest': '200.136.80.20', 'destSite': 'SPRACE', '_type': 'packetloss', 'packetLossAvg': 0.12956944444444446, 'alarmTime': 1471281866452, '_index': 'alarms-2016.08', 'srcSite': 'FZK-LCG2', 'src': '192.108.47.12', 'measurements': 6}
{'dest': '129.15.40.231', 'destSite': 'OU_OCHEP_SWT2', '_type': 'packetloss', 'packetLossAvg': 0.022925925925925926, 'alarmTime': 1471281866452, '_index': 'alarms-2016.08', 'srcSite': 'RAL-LCG2', 'src': '130.246.176.109', 'measurements': 9}
{'dest': '141.108.35.18', 'destSite': 'INFN-ROMA1', '_type': 'packetloss', 'pa

### write alarms to Elasticsearch

In [10]:
try:
   res = helpers.bulk(es, toAlertOn, raise_on_exception=True,request_timeout=60)
   print("inserted:",res[0], '\tErrors:',res[1])
except es_exceptions.ConnectionError as e:
   print('ConnectionError ', e)
except es_exceptions.TransportError as e:
   print('TransportError ', e)
except helpers.BulkIndexError as e:
   print(e[0])
   for i in e[1]:
      print(i)
except:
   print('Something seriously wrong happened.')

inserted: 205 	Errors: []
